<h1>Test L2R Model</h1>

In [1]:
import sys
sys.path.insert(1, '../code')

In [2]:
import os
import csv
import pickle
import pandas as pd

from nltk import TweetTokenizer

from boosting_ranker import * 
from utils import * 

<h2>Load Data</h2>

<h3>Documents</h3>

In [3]:
docs = load_docs("../data/ask_reddit_posts_augmented.jsonl", text_key = "full_text")

In [4]:
docids = [doc[0] for doc in docs]

<h3>Training Data</h3>

In [18]:
queries_to_relevance = load_true_relevance("../eval/reformated_relevance_scores.csv")
queries_to_relevance_test = {k:v for k,v in list(queries_to_relevance.items())[:5]}

<h2>Load IR System Infastructure</h2>

In [7]:
document_preprocessor = TweetTokenizer()

In [8]:
with open("../files/stopwords.txt") as f:
    words = f.read()
    stopwords = words.split()

In [9]:
with open("../system_components/post_index", "rb") as f:
    post_index = pickle.load(f)

with open("../system_components/comment_index", "rb") as f:
    comment_index = pickle.load(f)


with open("../system_componnents/full_text_index", "rb") as f:
    full_text_index = pickle.load(f)

In [ ]:
bm25 = BM25(full_text_index)
ranker = Ranker(full_text_index, document_preprocessor, stopwords, bm25)

<h2>Feature files</h2>

In [10]:
cross_encoder_scores = {}
with open("cross_encoder_scores.csv", "r") as f: 
    reader = csv.reader(f)
    for line in reader: 
        if 'docid' in line:
            continue
        cross_encoder_scores[int(line[0])] = float(line[1])
    

In [11]:
karma_scores = {}
with open("../data/post_karma.csv", "r") as f: 
    reader = csv.reader(f)
    for line in reader:
        if 'docid' in line:
            continue
        karma_scores[int(line[0])] = (float(line[1]), float(line[2]))

In [12]:
sentiment_scores = {}
with open("../data/doc_sentiment.csv", "r") as f:
    reader = csv.reader(f)
    for line in reader:
        if 'docid' in line:
            continue
        try:
            score = float(line[0])
        except:
            score = 0
        sentiment_scores[int(line[1])] = score

<h2>Feature Extraction</h2>

In [13]:
feature_extractor = XGBRankerFeatures(post_index, comment_index, {'post': {'b': 0.1, 'k1': 0.9, 'k3': 0.9}, 
                                                                  'comment': {'b': 0.1, 'k1': 0.9, 'k3': 0.9}}, 
                                      cross_encoder_scores, karma_scores, sentiment_scores)

In [14]:
query = 'what is something that is fun'

In [15]:
ranker = XGBRankerWrapper(feature_extractor, stopwords, document_preprocessor)

In [19]:
tup = ranker.fit(queries_to_relevance_test)

5it [00:40,  8.11s/it]

(140, 8) (140,) (140,)


In [22]:
ranker.query([1], 'is god real?', cutoff = 10)

AttributeError: 'XGBRankerWrapper' object has no attribute 'ranker'

In [ ]:
np.array(tup[1]).shape

In [ ]:
ranker.predict([2], 'i am just a girl')

In [ ]:
import xgboost as xgb
from sklearn.datasets import make_classification

# Make a synthetic ranking dataset for demonstration
seed = 1994
X, y = make_classification(random_state=seed)
rng = np.random.default_rng(seed)
n_query_groups = 3
qid = rng.integers(0, 3, size=X.shape[0])

In [ ]:
y